In [1]:
import numpy as np
import pandas as pd
import json
import os
import math

from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy,Dataset, Reader

In [2]:
ratings = pd.read_json(path_or_buf='../data/ratings.jsonl',lines=True)
#ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp']).astype(int)/10**9
#ratings['Timestamp'] = ratings['Timestamp']/(10**9)


/home/edu/miniconda3/envs/AI-ECG/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [3]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[['UserId', 'ItemId','Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

In [4]:
model = SVD()
model.fit(trainset)
print("training done")

training done


In [5]:
predictions = model.test(testset)
print("RMSE do modelo: "+str(accuracy.rmse(predictions)))

RMSE: 1.5152
RMSE do modelo: 1.5152330918343309


In [6]:
targets = pd.read_csv("../data/targets.csv")
targets["Rating"] = np.zeros(len(targets))
targets = targets.to_numpy()

predictions = model.test(targets)
predictions = [[tup.uid,tup.iid,tup.est] for tup in predictions]

In [7]:
predictions = pd.DataFrame(data=predictions,columns=['UserId', 'ItemId','Rating'])
predictions = predictions.sort_values(['UserId', 'Rating'], ascending=[True, False])
ratings = predictions["Rating"]
predictions = predictions.drop("Rating",axis=1)

predictions.to_csv("sub.csv",index=False)
    
print("submission done")

submission done
